In [2]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import scipy.stats as stats
import statsmodels.api as sm
from plotting_igor import PLOT
import glob
import openpyxl

In [3]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import PowerTransformer


from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet

# PARAMS

In [4]:
randomSeed = 0

# Import DF and Transform Prices


In [5]:
path = "../New_Data/4_df_man_noNanCat.csv"
df = pd.read_csv(path, sep = ",", index_col = 0) #index_col = 0

In [6]:
df

,Price,Age,Condition,Draught,Displacement,Certified No. of Persons,Engine Hours,No. of Cabins,No. of berths,No. of Bathrooms,...,Keel Type_Lifting Keel,Keel Type_Long keel,Keel Type_Swiveling Keel,Keel Type_Winged keel,Ship Area,image water,image land,image brightness,Latitude,Longitude
X,,,,,,,,,,,,,,,,,,,,,
0,15000,157,5.649642,0.450000,1.300000e+03,6.278333,903.797371,1.020000,2.010288,0,...,0,0,0,0,11.4912,1,1,94.002523,63.246778,25.920916
1,111300,149,5.332831,2.750000,1.620000e+05,14.420000,0.000000,5.524286,11.683917,0,...,0,0,0,0,145.5000,1,0,166.491938,46.603354,1.888334
2,252500,140,5.022946,2.000000,3.000000e+04,11.200000,236.327148,3.478889,7.990000,0,...,0,0,0,0,65.0590,1,0,163.811123,55.670249,10.333328
3,235400,131,5.649642,1.100000,2.933655e+06,10.960000,0.000000,3.340000,4.000000,0,...,0,0,0,0,115.0000,1,0,132.218456,63.246778,25.920916
4,81400,131,5.649642,0.500000,3.506178e+05,9.140000,903.797371,1.846667,4.000000,0,...,0,0,0,0,78.6900,1,0,142.688756,63.246778,25.920916
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13598,23200,-1,7.317987,0.429302,5.100000e+02,6.000000,2.623059,1.010000,1.960000,0,...,0,0,0,0,10.1920,1,0,130.589486,46.798562,8.231974
13599,256800,-1,7.317987,0.908169,9.736035e+03,8.000000,2.623059,2.000000,5.000000,0,...,0,0,0,0,34.3530,1,1,119.255349,46.798562,8.231974
13600,307300,-1,8.000000,2.200000,1.020000e+04,9.000000,5.385058,4.000000,7.080000,3,...,0,0,0,0,61.8995,1,1,120.430505,46.603354,1.888334


In [7]:
missingDF = df[df['Price'] <= 500000].copy()

In [8]:
# df["Price"] = boxcox(df["Price"], 0.5)

# YEO JOHNSON

In [9]:
pt = PowerTransformer(method='yeo-johnson', standardize=True)  # standardize=True z-scores the output

firstDF = df.drop(['Price'], axis=1)
transformedDF = pt.fit_transform(firstDF)

# Create a DataFrame from the transformed data
transformedDF = pd.DataFrame(transformedDF, columns=firstDF.columns)


# MODEL BUILDING 

In [18]:
def ridge(data: pd.DataFrame, transformedDF: pd.DataFrame) -> dict:
    

    
    X = transformedDF
    # y = np.log(data['Price'])
    y = data['Price']
    rmse_list = []
    mae_list = []
    kFold = KFold(n_splits=5, shuffle=True, random_state= randomSeed)

    for train_index, test_index in kFold.split(X):
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        # threshold = np.quantile(y_train, 0.7)
        # weights = np.where(y_train > threshold, 2.0, 1.0)

        # Define the model
        model = Ridge(alpha = 10, random_state = randomSeed, fit_intercept = True)

        # Train the model on the training set
        model.fit(X_train, y_train)

        # Make predictions on the testing set
        predictions = model.predict(X_test)
        # predictions = np.exp(predictions)
        # y_test = np.exp(y_test)
        
        # Calculate and save the RMSE for this fold
        mse = mean_squared_error(y_test, predictions)
        rmse = np.sqrt(mse)
        rmse_list.append(rmse)

        # Calculate and save the MAE for this fold
        mae = mean_absolute_error(y_test, predictions)
        mae_list.append(mae)

    # for i, rmse in enumerate(rmse_list, 1):
    #     print(f"RMSE for fold {i}: {rmse}")

    # for i, mae in enumerate(mae_list, 1):
    #     print(f"MAE for fold {i}: {mae}")

    print(rmse_list)
    print(mae_list)

    average_rmse = np.mean(rmse_list)
    average_mae = np.mean(mae_list)
    print(f"Average RMSE over all folds: {average_rmse}")
    print(f"Average MAE over all folds: {average_mae}")
    
    return {"RMSE": average_rmse, "MAE": average_mae}
    
    



def lasso(data: pd.DataFrame, transformedDf: pd.DataFrame) -> dict:
    
    X = transformedDF
    # y = np.log(data['Price'])
    y = data['Price']
    rmse_list = []
    mae_list = []
    kFold = KFold(n_splits=5, shuffle=True, random_state= randomSeed)

    for train_index, test_index in kFold.split(X):
        print("YES WE ARE RUNNNINGGGG")
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]


        # Define the model
        model = Lasso(alpha = 0.0001, random_state = randomSeed, max_iter = 10000)

        # Train the model on the training set
        model.fit(X_train, y_train)

        # Make predictions on the testing set
        predictions = model.predict(X_test)
        # predictions = np.exp(predictions)
        # y_test = np.exp(y_test)
        
        # Calculate and save the RMSE for this fold
        mse = mean_squared_error(y_test, predictions)
        rmse = np.sqrt(mse)
        rmse_list.append(rmse)

        # Calculate and save the MAE for this fold
        mae = mean_absolute_error(y_test, predictions)
        mae_list.append(mae)

    # for i, rmse in enumerate(rmse_list, 1):
    #     print(f"RMSE for fold {i}: {rmse}")

    # for i, mae in enumerate(mae_list, 1):
    #     print(f"MAE for fold {i}: {mae}")

    print(rmse_list)
    print(mae_list)

    average_rmse = np.mean(rmse_list)
    average_mae = np.mean(mae_list)
    print(f"Average RMSE over all folds: {average_rmse}")
    print(f"Average MAE over all folds: {average_mae}")
    
    return {"RMSE": average_rmse, "MAE": average_mae}



def elastic(data: pd.DataFrame, transformedDF: pd.DataFrame) -> dict:
    X = transformedDF
    # y = np.log(data['Price'])
    y = data['Price']
    rmse_list = []
    mae_list = []
    kFold = KFold(n_splits=5, shuffle=True, random_state= randomSeed)

    for train_index, test_index in kFold.split(X):
        # print("YES WE ARE RUNNNINGGGG")
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]


        # Define the model
        model = ElasticNet(alpha = 0.001, l1_ratio = 1, random_state= 1, fit_intercept= True, max_iter = 10000)

        # Train the model on the training set
        model.fit(X_train, y_train)

        # Make predictions on the testing set
        predictions = model.predict(X_test)
        # predictions = np.exp(predictions)
        # y_test = np.exp(y_test)
        
        # Calculate and save the RMSE for this fold
        mse = mean_squared_error(y_test, predictions)
        rmse = np.sqrt(mse)
        rmse_list.append(rmse)

        # Calculate and save the MAE for this fold
        mae = mean_absolute_error(y_test, predictions)
        mae_list.append(mae)

    # for i, rmse in enumerate(rmse_list, 1):
    #     print(f"RMSE for fold {i}: {rmse}")

    # for i, mae in enumerate(mae_list, 1):
    #     print(f"MAE for fold {i}: {mae}")

    # print(rmse_list)
    # print(mae_list)

    average_rmse = np.mean(rmse_list)
    average_mae = np.mean(mae_list)
    print(f"Average RMSE over all folds: {average_rmse}")
    print(f"Average MAE over all folds: {average_mae}")
    
    return {"RMSE": average_rmse, "MAE": average_mae}
    
    
    
def regress(data: pd.DataFrame, transformedDF: pd.DataFrame) -> dict:
    X = transformedDF
    X = sm.add_constant(X)
    # y = np.log(data['Price'])
    y = data['Price']
    rmse_list = []
    mae_list = []
    kFold = KFold(n_splits=5, shuffle=True, random_state= randomSeed)
        
    for train_index, test_index in kFold.split(X):
        # print("YES WE ARE RUNNNINGGGG")
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        



        # Define the model
        model = sm.OLS(y_train, X_train).fit()

        # Make predictions on the testing set
        predictions = model.predict(X_test)
        # predictions = np.exp(predictions)
        # y_test = np.exp(y_test)
        
        # Calculate and save the RMSE for this fold
        mse = mean_squared_error(y_test, predictions)
        rmse = np.sqrt(mse)
        rmse_list.append(rmse)

        # Calculate and save the MAE for this fold
        mae = mean_absolute_error(y_test, predictions)
        mae_list.append(mae)

    # for i, rmse in enumerate(rmse_list, 1):
    #     print(f"RMSE for fold {i}: {rmse}")

    # for i, mae in enumerate(mae_list, 1):
    #     print(f"MAE for fold {i}: {mae}")

    # print(rmse_list)
    # print(mae_list)

    average_rmse = np.mean(rmse_list)
    average_mae = np.mean(mae_list)
    # print(f"Average RMSE over all folds: {average_rmse}")
    # print(f"Average MAE over all folds: {average_mae}")
    
    return {"RMSE": average_rmse, "MAE": average_mae}


        
     


# TESTING RESULTS

In [ ]:
ridge(missingDF, transformedDF)

In [ ]:
lasso(missingDF, transformedDF)

In [ ]:
elastic(missingDF, transformedDF)

In [ ]:
regress(missingDF, transformedDF)

# DATA TRANSFORMATIONS

In [11]:
def transform_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    pt = PowerTransformer(method='yeo-johnson', standardize=True)  # standardize=True z-scores the output

    firstDF = df.drop(['Price'], axis=1)
    transformedDF = pt.fit_transform(firstDF)

    # Create a DataFrame from the transformed data
    transformedDF = pd.DataFrame(transformedDF, columns=firstDF.columns)
    
    
    missingDF = df[df['Price'] <= 500000].copy()
    
    return transformedDF, missingDF

# LOOKING THROUGH EVERY DF

In [16]:
files = []

for i in range(2, 8):
    directory_path = f"../New_Data/{i}"
    pattern = directory_path + "*.csv"
    files.extend(glob.glob(pattern))
    print(glob.glob(pattern))



['../New_Data/2_df_man.csv']
['../New_Data/3_df_man_PCA.csv']
['../New_Data/4_df_man_noNanCat.csv']
['../New_Data/5_df_freq.csv']
['../New_Data/6_df_freq_PCA.csv']
['../New_Data/7_df_freq_noNanCat.csv']


In [ ]:
files = []

for i in range(2, 8):
    directory_path = f"../New_Data/{i}"
    pattern = directory_path + "*.csv"
    files.extend(glob.glob(pattern))

errors = {}

count = 2
for file in files:
    df = pd.read_csv(f"{file}", sep = ",", index_col = 0)
    transformedDF, missingDF = transform_dataframe(df)
    errors[f"df {count}"] = {}
    errors[f"df {count}"]["Ridge"] = ridge(missingDF, transformedDF)
    errors[f"df {count}"]["Lasso"] = lasso(missingDF, transformedDF)
    errors[f"df {count}"]["Elastic"] = elastic(missingDF, transformedDF)
    errors[f"df {count}"]["Regression"] = regress(missingDF, transformedDF)
    
    count += 1
    

In [20]:
errors_df = pd.DataFrame(errors)
errors_df

,df 2,df 3,df 4,df 5,df 6,df 7
Ridge,"{'RMSE': 61654.16770798988, 'MAE': 44511.20745...","{'RMSE': 62907.05266663312, 'MAE': 45790.32391...","{'RMSE': 61654.09261203297, 'MAE': 44511.10087...","{'RMSE': 62178.227441588984, 'MAE': 45230.8094...","{'RMSE': 62756.87450531898, 'MAE': 45813.95730...","{'RMSE': 62178.16272383352, 'MAE': 45230.78150..."
Lasso,"{'RMSE': 61579.46795560975, 'MAE': 44042.89043...","{'RMSE': 62701.35726200809, 'MAE': 44857.04521...","{'RMSE': 61579.46795228836, 'MAE': 44042.89043...","{'RMSE': 62016.18312316025, 'MAE': 44444.82653...","{'RMSE': 62539.756033889076, 'MAE': 44845.6780...","{'RMSE': 62016.18312335408, 'MAE': 44444.82653..."
Elastic,"{'RMSE': 61579.466085930486, 'MAE': 44042.8939...","{'RMSE': 62701.3564123821, 'MAE': 44857.051241...","{'RMSE': 61579.46605278681, 'MAE': 44042.89390...","{'RMSE': 62016.18228181263, 'MAE': 44444.83261...","{'RMSE': 62539.75519538801, 'MAE': 44845.68457...","{'RMSE': 62016.18228033617, 'MAE': 44444.83260..."
Regression,"{'RMSE': 61584.7255863169, 'MAE': 43931.330519...","{'RMSE': 62724.58897135543, 'MAE': 44775.08446...","{'RMSE': 61579.46888172922, 'MAE': 44042.88930...","{'RMSE': 62046.55134600114, 'MAE': 44391.46292...","{'RMSE': 62551.29038048025, 'MAE': 44761.08896...","{'RMSE': 62016.18362510963, 'MAE': 44444.82535..."


In [24]:
# Creating DataFrame for RMSE and MAE
df_rmse = errors_df.applymap(lambda x: x['RMSE'])
df_mae = errors_df.applymap(lambda x: x['MAE'])

/var/folders/1r/8kyrcg852fgf6nffdk5rw72w0000gn/T/ipykernel_68684/1460705025.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_rmse = errors_df.applymap(lambda x: x['RMSE'])
/var/folders/1r/8kyrcg852fgf6nffdk5rw72w0000gn/T/ipykernel_68684/1460705025.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_mae = errors_df.applymap(lambda x: x['MAE'])


In [25]:
# Reordering the rows to make 'Regression' the top row
df_rmse_reordered = df_rmse.reindex(["Regression", "Ridge", "Lasso", "Elastic"])
df_mae_reordered = df_mae.reindex(["Regression", "Ridge", "Lasso", "Elastic"])

In [26]:
path = "../New_Data/RMSE.xlsx"
df_rmse_reordered.to_excel(path)

In [27]:
df_rmse_reordered

,df 2,df 3,df 4,df 5,df 6,df 7
Regression,61584.725586,62724.588971,61579.468882,62046.551346,62551.290380,62016.183625
Ridge,61654.167708,62907.052667,61654.092612,62178.227442,62756.874505,62178.162724
Lasso,61579.467956,62701.357262,61579.467952,62016.183123,62539.756034,62016.183123
Elastic,61579.466086,62701.356412,61579.466053,62016.182282,62539.755195,62016.182280


In [28]:
df_rmse_reordered.to_clipboard(index=True, header=True)

In [29]:
path = "../New_Data/MAE.xlsx"
df_mae_reordered.to_excel(path)
df_mae_reordered

,df 2,df 3,df 4,df 5,df 6,df 7
Regression,43931.330519,44775.084461,44042.889302,44391.462930,44761.088960,44444.825351
Ridge,44511.207459,45790.323918,44511.100871,45230.809426,45813.957304,45230.781508
Lasso,44042.890433,44857.045210,44042.890430,44444.826533,44845.678005,44444.826534
Elastic,44042.893935,44857.051242,44042.893903,44444.832614,44845.684577,44444.832601


In [30]:
df_mae_reordered.to_clipboard(index=True, header=True)

# Find optimal parameters 

In [10]:
def ridge_check(data: pd.DataFrame, transformedDf: pd.DataFrame) -> None: 
    X = transformedDF
    y = np.log(data['Price'])

    rmse_list = []
    mae_list = []
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=randomSeed)
    
    ridge = Ridge()

    # Define a grid of parameters over which to optimize the ridge regressor
    parameters = {'alpha': [0.01, 0.1, 1.0, 10, 100]}  # Example parameter grid with discrete values

    grid_search = GridSearchCV(ridge, parameters, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)

    print("Best parameters found: ", grid_search.best_params_)
    print("Best score found: ", grid_search.best_score_)


def lasso_check(data: pd.DataFrame, transformedDF: pd.DataFrame) -> None:
    X = transformedDF
    y = np.log(data['Price'])

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=randomSeed)

    lasso = Lasso()

    # Define a grid of parameters over which to optimize the lasso regressor
    parameters = {'alpha': [0.001, 0.01, 0.1, 1, 10], 'max_iter': [10000]}  # Example parameter grid with discrete values

    grid_search = GridSearchCV(lasso, parameters, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)

    print("Best parameters found: ", grid_search.best_params_)
    print("Best score found: ", grid_search.best_score_)


def elastic_net_check(data: pd.DataFrame, transformedDF: pd.DataFrame) -> None:
    X = transformedDF
    y = np.log(data['Price'])

    # Split the dataset into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=randomSeed)
    
    elastic_net = ElasticNet()

    # Define a grid of parameters over which to optimize the elastic net regressor
    parameters = {
        'alpha': [0.01, 0.1, 1, 10, 100],
        'l1_ratio': [0.001, 0.01, 0.1, 1],  # Ranging from Ridge to Lasso
        'max_iter': [200000]  # Ensuring convergence
    }

    grid_search = GridSearchCV(elastic_net, parameters, cv=5, scoring='neg_mean_squared_error')
    grid_search.fit(X_train, y_train)

    print("Best parameters found: ", grid_search.best_params_)
    print("Best score found: ", grid_search.best_score_)